# GTI720 - Protection des renseignements personnels

## Laboratoire 3 : Geoprivacy

| NOMS                  | CODE PERMANENT                                   |
|-----------------------|--------------------------------------------------|
| Nicolas Beaulieu      | BEAN26099604                                     |
| Gregson Destin        | DESG75010004                                     |
| Joël Pimparé-Larocque | PIMJ64050000                                     |
| Gabriel Gérin-Roze    | GERG83040008                                     |

# Description
Le but de ce laboratoire est de vous familiariser avec l'analyse et les attaques par inférence sur des données géolocalisées. Plus précisément, vous allez travailler sur trois fichiers représentant les traces de mobilité de différents individus collectées pendant plusieurs mois. Pour deux d’entre eux, il s’agit de taxis de San Francisco, pour l’autre d’un individu Français. Il vous faudra analyser ces traces de mobilité en essayant d'y extraire des informations telles que les "hotspots" (points de la carte très fréquentées) et si c'est possible les points d'intérêts concernant les individus dont vous avez reçu les traces. Pour cela, vous utiliserez **Python dans ce notebook**, la visualisation de données géographiques, mais vous avez évidemment le droit d'utiliser d'autres sources publiques d'information (telles que PagesJaunes, YahooMaps ou Google Streetview par exemple).

# Partie 0: Setup

- Créer un environnement python nommé `geoprivacy` 
- Installer la librairie `scikit-mobility`: `pip install scikit-mobility` (Python 3.9 recommandé ou google colab)

In [2]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import skmob
import folium

In [3]:
def spacial_dist(lat_1, lng_1, lat_2, lng_2):
    """
    Calcule la distance entre deux points. Les points doivent être exprimés 
    en coordoonées GPS (float). La distance est exprimée en mètre.
    """
    if (lat_1 == lat_2 and lng_1 == lng_2): 
        return 0.0
    
    er = 6366707
    latFrom = np.radians(lat_1)
    latTo = np.radians(lat_2)
    lngFrom = np.radians(lng_1)
    lngTo = np.radians(lng_2)
    
    return np.arccos(
        np.sin(latFrom) * np.sin(latTo)+\
        np.cos(latFrom) * np.cos(latTo) * np.cos(lngTo - lngFrom)
        )* er

In [4]:
def time_diff(time_1, time_2):
    """
    Calcul de la différence entre deux valeurs de datetime. La différence 
    est exprimée en seconde.
    """
    
    return np.timedelta64(time_2 - time_1, 's').astype("int") if time_2 > time_1 else np.timedelta64(time_1 - time_2, "s").astype("int")

In [5]:
def calcul_vitesse_point(lat_1, lng_1, time_1, lat_2, lng_2, time_2):
    """
    Calcul la vitesse entre deux points.
    
    lat_1, lng_1, time_1 : latitude, longitude et temps du premier point
    lat_2, lng_2, time_2 : latitude, longitude et temps du second point 
    
    return: vit, la vitesse entre ces deux points en mètre/seconde.

    """
    return spacial_dist(lat_1, lng_1, lat_2, lng_2) / time_diff(time_1, time_2) if time_diff(time_1, time_2) > 0 else 0


In [6]:
# vectorisation de la focntion calcul_vitesse_point
f = np.vectorize(calcul_vitesse_point)

In [8]:
def plot_points(lat_array, lng_array, zoom=6):
    """ Affiche les points représentés par lat_array et lng_array sur une carte Folium.
    :lat_array: tableau des latitudes.
    :lng_array: tableau des longitudes.
    
    :return: une carte Folium des points.
    """
    
    lat_array = np.array(lat_array)
    lng_array = np.array(lng_array)
        
    assert lat_array.shape[0] == lng_array.shape[0], "Les tableaux de latitudes et longitudes ne sont pas de la meme taille !"
    
    size_a = lat_array.shape[0]
    center = [np.sum(lat_array)/size_a, np.sum(lng_array)/size_a]
    mymap = folium.Map(location=center, zoom_start=zoom, tiles='Stamen Toner')
    
    for i in range(0, size_a):
        folium.CircleMarker(location=[lat_array[i], lng_array[i]], radius=5, color='blue').add_to(mymap)
        
    return mymap

In [9]:
def compute_centroid(arr_lat, arr_lng):
    """
    arr_lat: tableau des latitudes
    arr_lng: tableau des longitudes
    """
    
    xx = np.cos(np.radians(arr_lat)) * np.cos(np.radians(arr_lng))
    yy = np.cos(np.radians(arr_lat)) * np.sin(np.radians(arr_lng))
    zz = np.sin(np.radians(arr_lat))

    xxx = xx.sum() / xx.shape[0]
    yyy = yy.sum() / xx.shape[0]
    zzz = zz.sum() / xx.shape[0]

    assert (xx.shape[0] == yy.shape[0] and xx.shape[0] == zz.shape[0])

    central_longitude = np.arctan2(yyy, xxx)
    central_square_root = np.sqrt(xxx * xxx + yyy * yyy)
    central_latitude = np.arctan2(zzz, central_square_root)

    return np.array([np.degrees(central_latitude), np.degrees(central_longitude)])

# Partie 1 : Visualisation des données 

Vous devez pour cette partie, lire et afficher ces traces de mobilités.

### Q1 [5pts]
- À partir des fichiers `ID1.csv`,`ID2.csv` et `ID3.csv` créer un dataframe dont les colonnes sont ["latitude", "longitude", "time", "user_id"]
- user_id: identifiant (1, 2, 3) selon l'utilisateur

In [33]:
df1 = pd.read_csv("ID1.csv")
df2 = pd.read_csv("ID2.csv")
df3 = pd.read_csv("ID3.csv")

df1["user_id"] = 1
df2["user_id"] = 2
df3["user_id"] = 3

df = pd.concat([df1, df2, df3], axis=0)
#cange column time to datetime
#df["time"] = pd.to_datetime(df["time"])

print(df.head())

   latitude  longitude        time  user_id  bool
0  43.44383    6.76728  1254493883        1   NaN
1  43.44383    6.76728  1254560773        1   NaN
2  43.48308    1.57029  1254562354        1   NaN
3  43.49010    1.56436  1254562414        1   NaN
4  43.50372    1.55787  1254562475        1   NaN


### Q2 [5 pts]

- À l'aide de la librairie `Scikit-Mobility` et du dataframe obtenu ci-haut, créer un `TrajDataFrame`
- Afficher les trajectoires TrajDataFrame ainsi crées 
- Documentation: [Scikit-Mobility](https://github.com/scikit-mobility/scikit-mobility#examples).

In [37]:
#À l'aide de la librairie `Scikit-Mobility` et du dataframe obtenu ci-haut, créer un `TrajDataFrame`

#traj = skmob.TrajDataFrame(df, columns=["user_id", "latitude", "longitude", "time"])
traj = skmob.TrajDataFrame(df, latitude="latitude", longitude="longitude", datetime="time", user_id="user_id")
traj.plot_trajectory(start_end_markers=True, zoom=6, opacity=0.8, weight=3)

c:\Users\nicni\Documents\ecole\GTI720\Donnees-de-localisation\.venv\lib\site-packages\skmob\core\trajectorydataframe.py:569: UserWarning: Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify the desired number of users, or filter the TrajDataFrame.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,
c:\Users\nicni\Documents\ecole\GTI720\Donnees-de-localisation\.venv\lib\site-packages\skmob\core\trajectorydataframe.py:569: UserWarning: If necessary, trajectories will be down-sampled to have at most `max_points` points. To avoid this, specify `max_points=None`.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,


### Q3 [10pts]

À partir d’une simple visualisation (on peut bien entendu zoomer), que pouvez vous inférer de ces traces de mobilité ?

Nous pouvons inférer que les utilisateur 2 et 3 habite la même ville, ils partages certaines routes et on possiblement des rencontres.

# Partie 2 :Attaques par inférence sur données de mobilité

Vous devez pour cette partie, implémenter les deux attaques par inférence qui sont décrites ci-après. 

### Q4 [30 pts]
- Implémenter une attaque de type « BeginEnd » qui essaye d'identifier les points d'intérêts d'un individu en découvrant des "trous" dans ses traces de mobilité et en considérant le point d'arrivée et de départ avant ce trou comme des points d'intérêts possibles. Une heuristique simple pour trouver ces "trous" est de mesurer la vitesse atteinte à chaque point et de ne garder que les points où la vitesse de l'utilisateur est en dessous d'un seuil (3km/h par exemple).
- Visualiser le résultat

#### Aide

- Code: Nous vous fournissons pour ces questions les fonctions pour effectuer:
    - le calcul de la distance entre deux points GPS: `spacial_dist`
    - la différence entre deux informations temporelles (objet datetime): `time_diff`
    - la vitesse d'un objet entre deux points: `calcul_vitesse_point`
    - l'affichage des points d'arrêts: `plot_points`
    - Ces fonctions peuvent-être utilisées telles quelles, en partie ou pas du tout.

In [47]:
#Implémenter une attaque de type « BeginEnd » qui essaye d'identifier les points d'intérêts d'un individu en découvrant des "trous" dans ses traces de mobilité et en considérant le point d'arrivée et de départ avant ce trou comme des points d'intérêts possibles. Une heuristique simple pour trouver ces "trous" est de mesurer la vitesse atteinte à chaque point et de ne garder que les points où la vitesse de l'utilisateur est en dessous d'un seuil (3km/h par exemple).
#Code: Nous vous fournissons pour ces questions les fonctions pour effectuer:
    #- le calcul de la distance entre deux points GPS: `spacial_dist`
    #- la différence entre deux informations temporelles (objet datetime): `time_diff`
    #- la vitesse d'un objet entre deux points: `calcul_vitesse_point`
    #- l'affichage des points d'arrêts: `plot_points`
    #- Ces fonctions peuvent-être utilisées telles quelles, en partie ou pas du tout.

def find_POIs(dt, seuil=3, ):
    """
    dr: DataFrame
    seuil: vitesse en km/h
    """

    #liste des points d'arrêt
    poi = []
    for i in range(1, dt.shape[0]):
        vitesse = calcul_vitesse_point(dt["latitude"][i-1], dt["longitude"][i-1], dt["time"][i-1], dt["latitude"][i], dt["longitude"][i], dt["time"][i])
        if vitesse < seuil:
            #ajout seulement des latitudes et longitudes des points d'arrêt
            poi.append([dt["latitude"][i-1], dt["longitude"][i-1]])
            poi.append([dt["latitude"][i], dt["longitude"][i]])
    return np.array(poi)

### Format des résultats

In [43]:
# exemple montrant comment sélectionner les points d'un seul individus.
# Remplacez 1 par 2 ou 3 pour les autres individus.
points_id1 = df.loc[df.user_id == 1]

In [48]:
# Calculez les points d'arrêts ici, vous devez évidemment changer les valeurs des tableaux
# ci-dessous qui ne servent qu'à illustrer le type de résultats à fournir.

stop_point_id1 = find_POIs(df.loc[df.user_id == 1]) # resultat ici de la forme d'un tableau de points :
                                                        # [[x,y], [x', y'], ..., [x'', y'']] 
                                                        # avec x latitude et y longitude. À modifer
        
stop_point_id2 = np.array([0,1,2,3,4,5]).reshape((3,2)) # idem. À modifier
stop_point_id3 = np.array([0,1,2,3,4,5]).reshape((3,2)) # idem. À modifier

KeyError: 'datetime'

In [42]:
# votre tableau de points d'arrêts doit avoir au moins deux colonnes
# correspondant à la latitude et la longitude des points d'arrêts.

assert stop_point_id1.shape[1] >= 2, "Vous n'avez pas assez de colonnes dans votre tableau de résultats"

In [ ]:
### Exemple d'affichage de points d'arrêts de l'utilisateur 1 (ce ne sont pas ses vraies coordonnées)

# plot(stop_point_id1)
plot_points(stop_point_id1[:,0], stop_point_id1[:,1]) # ici [:,0] signifie que l'on sélectionne
                                                      # toutes les données de la première colonne
                                                      # de stop_points_id1 (idem pour [:,1] avec
                                                      # la seconde colonne).

### Q5 [30 pts]
- Implémenter une attaque qui utilise un algorithme de clustering du type k-moyennes (kmean) pour essayer de trouver des groupes de localisation dont la localisation moyenne ou médianes pourrait correspondre à un point d'intérêt (par exemple la maison). Cet algorithme pourra prendre en entrée soit la trace totale, soit le résultat de  l'attaque précédente et retournera en sortie une représentation des différents groupes (clusters) découverts.
- Visualiser le résultat

#### Aide

- Pour cette question nous vous proposons d'utiliser l'algorithme [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) pour clusteriser les points. D'[autres algorithmes](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster) de clustering existent (p. e. [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)). Vous êtes invités à les expérimenter, afin de voir les différences de résultats, mais ce n'est pas obligatoire. Pour plus d'informations sur la clusterisation de points GPS vous pouvez consulter [ce tutoriel](https://chih-ling-hsu.github.io/2018/01/02/clustering-python).


In [11]:
from sklearn.cluster import DBSCAN

- Pour mesurer les distances entre des points GPS il faut d'abord les convertir en radians. Pour ça vous pouvez utiliser la [fonction](https://numpy.org/doc/stable/reference/generated/numpy.radians.html) `radians()` de `numpy`.

In [12]:
X = stop_point_id1 # convertir en radians, à modifier

- DBSCAN dispose de plusieurs paramètres :

    - le paramètre epsilon, il définit la distance maximale pour que deux points soient considérés comme appartenant au même cluster.

    - le paramètre min-sample, il définit le nombre de point minimum qui doivent se trouver dans un rayon epsilon d'un point pour que celui-ci soit considéré comme un point noyau.

    - l'algorithme d'indexation utilisé par DBSCAN pour clusteriser les points (ici ball_tree). C'est un paramètre d'optimisation pour le calcul des clusters.

    - la métrique à utiliser comme distance entre les points. Ici nous utilisons haversine qui est une distance pour les points GPS (la terre étant ronde, l'utilisation de la distance euclidienne est à bannir).
    
    - Comme les points sont exprimés en radians, la distance epsilon doit être exprimée en radians aussi. Il faut alors la diviser par le rayon de la terre, soit environ 6366,707km (évidemment epsilon doit être exprimé dans la même unité que celle du rayon de la terre).

In [13]:
eps = 1         # à modifier
min_samples = 0  # à modifier
db = DBSCAN(eps=eps, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(X)

In [ ]:
print("Nombre de clusters créer : ", len(set(db.labels_)) - 1)

- Nous vous fournissons aussi deux fonctions qui:

    - étant donnée un cluster de points retourne le centroids de ce cluster: `compute_centroid`
    - et la fonction pour l'affichage des points d'arrêts: `plot_points`


#### Format des résultats

In [16]:
# Attention ici nous calculons le centroid **D'UN SEUL** cluster de points.
# Il y a plusieurs clusters de points par individu et plusieurs individus (id1, id2 et id3).

lat_cluster_1 = np.array([0,2,4]) # à modifier
lng_cluster_1 = np.array([1,3,5]) # à modifier
centroid = compute_centroid(lat_cluster_1, lng_cluster_1) # calcul du centroid d'un cluster de points inventés

#### Affichage

Pour chaque fichier (id) afficher les centroids des clusters des points d'arrêts

In [17]:
# plot(stop_point_id1)
# plot(stop_point_id2)
# plot(stop_point_id3)

### Q6 [10 pts]
- Que pouvez vous déduire sur l'utilisateur 1, l'utilisateur 2 et l'utilisateur 3 à partir des résultats de la question 4 ?

### Q7 [10 pts]
- Que pouvez vous déduire sur l'utilisateur 1, l'utilisateur 2 et l'utilisateur 3 à partir des résultats de la question 5 ?